In [1]:
import os
import pandas as pd
import numpy as np

/var/folders/6y/6p7qrh9s27d1s3l8p2l81vbw0000gn/T/ipykernel_76081/1576438456.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
data = dict()
for year in range(2002, 2025):
    year_data = dict()
    for dirname, _, filenames in os.walk(f'../data/kenpom/{year}'):
        for filename in filenames:
            if filename == ".DS_Store":
                continue
            table_name = filename.split('.')[0]
            bic = table_name.split(".csv")[0].split("/")[-1][:-2] #ball in court looking for offense/defense
            table_path = os.path.join(dirname, filename)
            try:
                temp = pd.read_csv(table_path)
                temp = temp.rename(columns={col:col.replace("_", "") for col in temp.columns})
                if (bic == "offense") | (bic == "defense"):
                    temp = temp.rename(columns={col: col + "_" + bic for col in temp.columns if col not in ["Season", "TeamName"]})
                year_data[table_path] = temp
            except UnicodeDecodeError:
                temp = pd.read_csv(table_path, encoding='cp1252')
                temp = temp.rename(columns={col:col.replace("_", "") for col in temp.columns})
                if (bic == "offense") | (bic == "defense"):
                    temp = temp.rename(columns={col: col + "_" + bic for col in temp.columns if col not in ["Season", "TeamName"]})
                year_data[table_path] = temp
            except Exception as e:
                print(f"Error with {filename}: {e}")
    data[year] = year_data
    df = pd.concat(data[year].values()).groupby('TeamName', as_index=False).first()
    data[year] = df


/var/folders/6y/6p7qrh9s27d1s3l8p2l81vbw0000gn/T/ipykernel_76081/3354023943.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df = pd.concat(data[year].values()).groupby('TeamName', as_index=False).first()
/var/folders/6y/6p7qrh9s27d1s3l8p2l81vbw0000gn/T/ipykernel_76081/3354023943.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df = pd.concat(data[year].values()).groupby('TeamName', as_index=False).first()
/var/folders/6y/6p7qrh9s27d1s3l8p2l81vbw0000gn/T/ipykernel_76081/3354023943.py:30: PerformanceWarning: DataFrame is 

Error with summary20.csv: No columns to parse from file


/var/folders/6y/6p7qrh9s27d1s3l8p2l81vbw0000gn/T/ipykernel_76081/3354023943.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df = pd.concat(data[year].values()).groupby('TeamName', as_index=False).first()
/var/folders/6y/6p7qrh9s27d1s3l8p2l81vbw0000gn/T/ipykernel_76081/3354023943.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df = pd.concat(data[year].values()).groupby('TeamName', as_index=False).first()
/var/folders/6y/6p7qrh9s27d1s3l8p2l81vbw0000gn/T/ipykernel_76081/3354023943.py:30: PerformanceWarning: DataFrame is 

In [3]:
# data[2002][[col for col in data[2002].columns if col.endswith("_offense")] + ["TeamName"]]

In [4]:
df = pd.concat(data.values(),ignore_index=True)

In [8]:
df["TeamName"] = df["TeamName"].str.replace(".", "")
df["TeamName"] = df["TeamName"].str.replace("Saint Mary's", "St Mary's CA")

In [11]:
df["TeamName"]

0           Air Force
1               Akron
2             Alabama
3         Alabama A&M
4          Alabama St
            ...      
8294        Wright St
8295          Wyoming
8296           Xavier
8297             Yale
8298    Youngstown St
Name: TeamName, Length: 8299, dtype: object

In [10]:
df.to_csv("../data/kenpom/kenpom_all.csv", index=False)